# Alpha Vantage API Demo

This notebook demonstrates how to use the Alpha Vantage client for fetching financial data, fundamentals, and news & sentiment analysis.

In [5]:
import sys
import os
import pandas as pd
import matplotlib.pyplot as plt

# Add the project root to the path to enable imports
sys.path.append(os.path.abspath('../..'))

# Import the Alpha Vantage client
from investment_machine.src.tools.data_fetchers.alpha_vantage import AlphaVantageClient

/Users/paggrawal/hobby_projects/trader/investment_machine/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [4]:
!pip3 install -q pandas matplotlib

You should consider upgrading via the '/Users/paggrawal/hobby_projects/trader/investment_machine/venv/bin/python3 -m pip install --upgrade pip' command.


## Initialize the Alpha Vantage Client

You can provide your API key directly or set it as an environment variable `ALPHA_VANTAGE_API_KEY`.

In [6]:
# Option 1: Read from environment variable
# client = AlphaVantageClient()

# Option 2: Provide API key directly
client = AlphaVantageClient(api_key="JTPKOHNIT0NJYEQ2")

## Fetching Daily Stock Data

Get daily adjusted price data for an Indian stock (using NSE/BSE extensions)

In [7]:
# Fetch daily adjusted data for Reliance Industries from NSE
reliance_daily = client.get_daily_adjusted("RELIANCE.NS")

# Display the last 5 data points
reliance_daily.tail()

""


In [8]:
# Plot the closing price
plt.figure(figsize=(12, 6))
plt.plot(reliance_daily.index, reliance_daily['adjusted close'])
plt.title('Reliance Industries - Adjusted Close Price')
plt.xlabel('Date')
plt.ylabel('Price (INR)')
plt.grid(True)
plt.show()

KeyError: 'adjusted close'

<Figure size 1200x600 with 0 Axes>

## Fetching Fundamental Data

Retrieve company overview and financial statements

In [9]:
# Get company overview
overview = client.get_company_overview("RELIANCE.NS")

# Display selected information
selected_info = {
    key: overview.get(key) for key in [
        "Symbol", "Name", "Description", "Sector", "Industry", "MarketCapitalization",
        "PERatio", "PEGRatio", "BookValue", "DividendYield", "EPS", "ProfitMargin", "ROE"
    ]
}

pd.Series(selected_info)

Symbol                  None
Name                    None
Description             None
Sector                  None
Industry                None
MarketCapitalization    None
PERatio                 None
PEGRatio                None
BookValue               None
DividendYield           None
EPS                     None
ProfitMargin            None
ROE                     None
dtype: object

In [ ]:
# Get income statement
income_statement = client.get_income_statement("RELIANCE.NS")

# Extract annual reports and create a DataFrame
annual_reports = income_statement.get("annualReports", [])
if annual_reports:
    # Create a DataFrame with key metrics
    metrics = [
        "fiscalDateEnding", "totalRevenue", "grossProfit", "operatingIncome", 
        "netIncome", "ebitda"
    ]
    
    # Extract data for each year
    data = []
    for report in annual_reports:
        data.append({metric: report.get(metric) for metric in metrics})
        
    # Create and display DataFrame
    income_df = pd.DataFrame(data)
    income_df.set_index("fiscalDateEnding", inplace=True)
    
    # Convert to numeric
    income_df = income_df.apply(pd.to_numeric)
    
    # Display in millions
    (income_df / 1_000_000).round(2)

## Calculate Key Financial Metrics

Use the helper method to calculate key financial metrics

In [ ]:
# Calculate key metrics for Reliance
metrics = client.calculate_key_metrics("RELIANCE.NS")

# Display metrics
pd.Series(metrics)

## News & Sentiment Analysis

Retrieve recent news and sentiment data

In [ ]:
# Get news for Reliance, limit to 10 items
news = client.get_news_sentiment(symbol="RELIANCE.NS", limit=10)

# Process and display news items
news_items = news.get("feed", [])
if news_items:
    # Create a list of news with relevant information
    processed_news = []
    for item in news_items:
        processed_news.append({
            "title": item.get("title"),
            "time_published": item.get("time_published"),
            "sentiment": item.get("overall_sentiment_label"),
            "sentiment_score": item.get("overall_sentiment_score"),
            "url": item.get("url")
        })
    
    # Create and display DataFrame
    news_df = pd.DataFrame(processed_news)
    news_df

## Topic-based News

Retrieve news based on specific topics relevant to Indian markets

In [ ]:
# Get news for specific topics (e.g., economy, ipo, technology)
topic_news = client.get_news_sentiment(topics="economy,ipo,technology", limit=10)

# Process and display topic-based news
topic_items = topic_news.get("feed", [])
if topic_items:
    # Create a list of news with relevant information
    processed_topics = []
    for item in topic_items:
        processed_topics.append({
            "title": item.get("title"),
            "time_published": item.get("time_published"),
            "topics": ", ".join([topic.get("topic") for topic in item.get("topics", [])]),
            "sentiment": item.get("overall_sentiment_label"),
            "url": item.get("url")
        })
    
    # Create and display DataFrame
    topics_df = pd.DataFrame(processed_topics)
    topics_df